In [2]:
import torch

from transformers import EsmModel, AutoTokenizer
from peft import LoraConfig, get_peft_model

/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# "facebook/esm2_t33_650M_UR50D"
tokenizer = AutoTokenizer.from_pretrained("../data/esm2-650M")
model = EsmModel.from_pretrained("../data/esm2-650M")

In [5]:
model.save_pretrained("../data/esm2-650M",) 
tokenizer.save_pretrained("../data/esm2-650M",)

('../data/esm2-650M/tokenizer_config.json',
 '../data/esm2-650M/special_tokens_map.json',
 '../data/esm2-650M/vocab.txt',
 '../data/esm2-650M/added_tokens.json')

{"timestamp":"2025-12-08T12:46:21.753669Z","level":"ERROR","fields":{"message":"error downloading range, error: ReqwestMiddlewareError(Middleware(error sending request\n\nCaused by:\n    0: client error (SendRequest)\n    1: connection error\n    2: cannot decrypt peer's message))","caller":"/Users/runner/work/xet-core/xet-core/cas_client/src/download_utils.rs:528"},"filename":"/Users/runner/work/xet-core/xet-core/error_printer/src/lib.rs","line_number":28}


In [5]:
seq = ["SEQVENCE", "ANOTHERSEQ"]

In [6]:
inputs = tokenizer(seq, return_tensors="pt", truncation=True, padding='max_length', max_length=100)
with torch.no_grad():
    output = model(**inputs)

In [7]:
output.last_hidden_state.shape

torch.Size([2, 100, 1280])

In [21]:
inputs

{'input_ids': tensor([[ 0,  8,  9, 16,  7,  9, 17, 23,  9,  2,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 0,  5, 17, 28, 11, 21,  9, 10,  8,  9, 16,  2,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [26]:
peft_config = LoraConfig(
            task_type="FEATURE_EXTRACTION",
            inference_mode=False,
            r=4,
            lora_alpha=1,
            bias="all",
            target_modules=["query", "key", "value", "fc1", "fc2"],
            #lora_dropout=0.1,
        )
model.train()
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 67,841 || all params: 7,557,881 || trainable%: 0.8976
